# Cart Pole with Policy Gradient

[reference](https://medium.com/@awjuliani/super-simple-reinforcement-learning-tutorial-part-2-ded33892c724)

[ *policy based agent*, *cart pole game*, *delayed reward* ]

In [1]:
import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import math